In [91]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.initializers import GlorotUniform, GlorotNormal

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
baseDir = '/content/drive/MyDrive/PADP/StatisticallearningII/Proyecto final/'
NIHCCdata = baseDir + 'four_findings_expert_labels/'
SampleData = baseDir + 'CXR8/samples/'

In [118]:
data_train = pd.read_csv(baseDir+'./four_findings_expert_labels/four_findings_expert_labels_validation_labels.csv')
print(data_train.shape)
data_train.head()

(2414, 16)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Fracture,Pneumothorax,Airspace opacity,Nodule or mass,Set Id
0,00000119_001.png,Atelectasis,1,119,56,M,AP,2500,2048,0.168,0.168,NO,NO,NO,NO,val
1,00000134_000.png,Infiltration,0,134,76,F,AP,2048,2500,0.168,0.168,NO,NO,YES,NO,val
2,00000135_001.png,No Finding,1,135,70,M,AP,2500,2048,0.168,0.168,NO,NO,YES,NO,val
3,00000156_000.png,Cardiomegaly|Infiltration,0,156,81,M,AP,2500,2048,0.171,0.171,NO,NO,YES,NO,val
4,00000156_001.png,Cardiomegaly|Effusion,1,156,81,M,AP,2500,2048,0.171,0.171,NO,NO,YES,NO,val


In [119]:
data_test = pd.read_csv(baseDir+'./four_findings_expert_labels/four_findings_expert_labels_test_labels.csv')
print(data_test.shape)
data_test.head()

(1962, 16)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Fracture,Pneumothorax,Airspace opacity,Nodule or mass,Set Id
0,00000013_008.png,No Finding,8,13,60,M,AP,3056,2544,0.139,0.139,NO,NO,YES,NO,test
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,57,M,AP,2500,2048,0.168,0.168,NO,YES,NO,NO,test
2,00000032_002.png,No Finding,2,32,55,F,AP,2500,2048,0.168,0.168,NO,NO,YES,NO,test
3,00000032_009.png,No Finding,9,32,55,F,AP,2500,2048,0.168,0.168,NO,NO,YES,NO,test
4,00000032_011.png,Infiltration,11,32,55,F,AP,2500,2048,0.168,0.168,NO,NO,YES,NO,test


In [121]:
data_trainN = data_train
data_trainN['NumFinding'] = data_trainN['Finding Labels'].apply(lambda x: len(x.split('|')))
data_trainN['Finding Labels'] = data_trainN['Finding Labels'].apply(lambda x: x.split('|')[0])
data_trainN['Patient Gender'] .replace('F', 0,inplace=True)
data_trainN['Patient Gender'] .replace('M', 1,inplace=True)
data_trainN['Fracture'] .replace('NO', 0,inplace=True)
data_trainN['Fracture'] .replace('YES', 1,inplace=True)
data_trainN['Pneumothorax'] .replace('NO', 0,inplace=True)
data_trainN['Pneumothorax'] .replace('YES', 1,inplace=True)
data_trainN['Airspace opacity'] .replace('NO', 0,inplace=True)
data_trainN['Airspace opacity'] .replace('YES', 1,inplace=True)
data_trainN['Nodule or mass'] .replace('NO', 0,inplace=True)
data_trainN['Nodule or mass'] .replace('YES', 1,inplace=True)
#data_trainN['Patient Age'] = data_trainN['Patient Age'].map(lambda x: str(x)[:-1]).astype(int)

data_trainN['Finding Labels'].value_counts().head(50)


No Finding            1320
Atelectasis            276
Infiltration           264
Effusion               172
Consolidation           89
Edema                   86
Nodule                  60
Mass                    51
Cardiomegaly            38
Emphysema               23
Pneumothorax            15
Pleural_Thickening      12
Fibrosis                 5
Pneumonia                3
Name: Finding Labels, dtype: int64

In [99]:
data_trainN.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Fracture,Pneumothorax,Airspace opacity,Nodule or mass,Set Id,NumFinding
0,00000119_001.png,Atelectasis,1,119,56,1,AP,2500,2048,0.168,0.168,0,0,0,0,val,1
1,00000134_000.png,Infiltration,0,134,76,0,AP,2048,2500,0.168,0.168,0,0,1,0,val,1
2,00000135_001.png,No Finding,1,135,70,1,AP,2500,2048,0.168,0.168,0,0,1,0,val,1
3,00000156_000.png,Cardiomegaly,0,156,81,1,AP,2500,2048,0.171,0.171,0,0,1,0,val,1
4,00000156_001.png,Cardiomegaly,1,156,81,1,AP,2500,2048,0.171,0.171,0,0,1,0,val,1


In [100]:
data_testN = data_test
data_testN['NumFinding'] = data_testN['Finding Labels'].apply(lambda x: len(x.split('|')))
data_testN['Finding Labels'] = data_testN['Finding Labels'].apply(lambda x: x.split('|')[0])
data_testN['Patient Gender'] .replace('F', 0,inplace=True)
data_testN['Patient Gender'] .replace('M', 1,inplace=True)
data_testN['Fracture'] .replace('NO', 0,inplace=True)
data_testN['Fracture'] .replace('YES', 1,inplace=True)
data_testN['Pneumothorax'] .replace('NO', 0,inplace=True)
data_testN['Pneumothorax'] .replace('YES', 1,inplace=True)
data_testN['Airspace opacity'] .replace('NO', 0,inplace=True)
data_testN['Airspace opacity'] .replace('YES', 1,inplace=True)
data_testN['Nodule or mass'] .replace('NO', 0,inplace=True)
data_testN['Nodule or mass'] .replace('YES', 1,inplace=True)

data_testN['Finding Labels'].value_counts().head(50)

No Finding            770
Atelectasis           240
Infiltration          228
Effusion              204
Consolidation         103
Cardiomegaly           79
Pneumothorax           77
Mass                   66
Emphysema              47
Edema                  46
Nodule                 44
Pleural_Thickening     22
Fibrosis               22
Pneumonia              10
Hernia                  4
Name: Finding Labels, dtype: int64

In [101]:
data_testN.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Fracture,Pneumothorax,Airspace opacity,Nodule or mass,Set Id,NumFinding
0,00000013_008.png,No Finding,8,13,60,1,AP,3056,2544,0.139,0.139,0,0,1,0,test,1
1,00000013_026.png,Cardiomegaly,26,13,57,1,AP,2500,2048,0.168,0.168,0,1,0,0,test,2
2,00000032_002.png,No Finding,2,32,55,0,AP,2500,2048,0.168,0.168,0,0,1,0,test,1
3,00000032_009.png,No Finding,9,32,55,0,AP,2500,2048,0.168,0.168,0,0,1,0,test,1
4,00000032_011.png,Infiltration,11,32,55,0,AP,2500,2048,0.168,0.168,0,0,1,0,test,1


In [102]:
data_trainN.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Fracture',
       'Pneumothorax', 'Airspace opacity', 'Nodule or mass', 'Set Id',
       'NumFinding'],
      dtype='object')

In [103]:
data_trainN.drop(['Patient ID', 'Image Index','View Position','OriginalImage[Width','Set Id',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'], axis=1, inplace=True)
data_trainN.shape

(2414, 9)

In [104]:
data_trainN.head()

,Finding Labels,Follow-up #,Patient Age,Patient Gender,Fracture,Pneumothorax,Airspace opacity,Nodule or mass,NumFinding
0,Atelectasis,1,56,1,0,0,0,0,1
1,Infiltration,0,76,0,0,0,1,0,1
2,No Finding,1,70,1,0,0,1,0,1
3,Cardiomegaly,0,81,1,0,0,1,0,1
4,Cardiomegaly,1,81,1,0,0,1,0,1


In [105]:
data_testN.drop(['Patient ID', 'Image Index','View Position','OriginalImage[Width','Set Id',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'], axis=1, inplace=True)
data_testN.shape

(1962, 9)

In [106]:
data_testN.head()

,Finding Labels,Follow-up #,Patient Age,Patient Gender,Fracture,Pneumothorax,Airspace opacity,Nodule or mass,NumFinding
0,No Finding,8,60,1,0,0,1,0,1
1,Cardiomegaly,26,57,1,0,1,0,0,2
2,No Finding,2,55,0,0,0,1,0,1
3,No Finding,9,55,0,0,0,1,0,1
4,Infiltration,11,55,0,0,0,1,0,1


In [53]:
y_train = data_trainN['Finding Labels']

In [56]:
print(y_train.shape)
y_train.head()

(2414,)


0     Atelectasis
1    Infiltration
2      No Finding
3    Cardiomegaly
4    Cardiomegaly
Name: Finding Labels, dtype: object

In [61]:
y_test = data_testN['Finding Labels']

In [62]:
print(y_test.shape)
y_test.head()

(1962,)


0      No Finding
1    Cardiomegaly
2      No Finding
3      No Finding
4    Infiltration
Name: Finding Labels, dtype: object

In [65]:
label_encoder = LabelEncoder()
y_real_train = label_encoder.fit_transform(y_train)
y_real_train = y_real_train.reshape(-1, 1)

In [66]:
y_real_train

array([[0],
       [7],
       [9],
       ...,
       [7],
       [7],
       [2]])

In [68]:
label_encoder = LabelEncoder()
y_real_test = label_encoder.fit_transform(y_test)
y_real_test = y_real_test.reshape(-1, 1)

In [69]:
y_real_test

array([[10],
       [ 1],
       [10],
       ...,
       [10],
       [10],
       [10]])

In [107]:
data_trainN.drop(['Finding Labels'], axis=1, inplace=True)
data_trainN.head()

,Follow-up #,Patient Age,Patient Gender,Fracture,Pneumothorax,Airspace opacity,Nodule or mass,NumFinding
0,1,56,1,0,0,0,0,1
1,0,76,0,0,0,1,0,1
2,1,70,1,0,0,1,0,1
3,0,81,1,0,0,1,0,1
4,1,81,1,0,0,1,0,1


In [108]:
data_testN.drop(['Finding Labels'], axis=1, inplace=True)
print(data_testN.shape)
data_testN.head()

      Follow-up #  Patient Age  ...  Nodule or mass  NumFinding
0               8           60  ...               0           1
1              26           57  ...               0           2
2               2           55  ...               0           1
3               9           55  ...               0           1
4              11           55  ...               0           1
...           ...          ...  ...             ...         ...
1957            1           36  ...               0           1
1958            2           48  ...               0           1
1959            0           54  ...               1           1
1960            0           64  ...               0           1
1961            0           38  ...               0           1

[1962 rows x 8 columns]


,Follow-up #,Patient Age,Patient Gender,Fracture,Pneumothorax,Airspace opacity,Nodule or mass,NumFinding
0,8,60,1,0,0,1,0,1
1,26,57,1,0,1,0,0,2
2,2,55,0,0,0,1,0,1
3,9,55,0,0,0,1,0,1
4,11,55,0,0,0,1,0,1


In [109]:
print(data_testN.shape)


(1962, 8)


In [110]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

In [111]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [92]:
X = data_trainN.to_numpy()
X = np.asarray(X).astype('float32')

ValueError: ignored

In [112]:
history = model.fit(data_trainN.to_numpy(), y_train,validation_split=0.3, epochs=20, batch_size=64)
model.save(baseDir+'CXR8/models/mlp.h5')

Epoch 1/20


UnimplementedError: ignored